# Queens Problem

### importem llibreries que utilitzarem i definim variables globals


In [10]:
import numpy as np
import random

INITIAL_POPULATION = 10
BOARD_SIZE = 8
MUTATION_CHANCE = 0.1

### Definir Agent, població i display

In [11]:
# Inicialitzar un gen amb una llista de números aleatoris de l'1 a la variable global BOARD_SIZE
def init_agent(board_size):
    agent = list(range(1, board_size + 1))
    random.shuffle(agent)
    return agent

# Funció que imprimeix un array com si fós un tauler d'escacs amb les reines al seu lloc
def display_agent(agent): 
    for queenPosition in agent:
        result = []

        for el in range(len(agent)):
            result.append(1 if el == queenPosition else 0)

        print(result)

# Funció per inicialitzar un conjunt de gens (població) segons la variable BOARD_SIZE
def init_population(initial_population, board_size):
    pop = []
    for _ in range(initial_population):
        pop.append(init_agent(board_size))
    return pop

_population = init_population(INITIAL_POPULATION, BOARD_SIZE)
_agent = _population[0]
assert BOARD_SIZE == len(_agent)
assert INITIAL_POPULATION == len(_population)
display_agent(_agent)
# assert BOARD_SIZE*BOARD_SIZE == len(display_agent(_agent))

[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]


### definir crossover

In [12]:
# Funció que rep dos gens i els reprodueix a partir de la meitat del gen
def crossover(parent1, parent2):
    # Crosspoint a la meitat 
    midpoint = len(parent1) // 2
    
    # Agafem la primera meitat del gen i li afegim els números que faltin de l'altre gen de manera que no hi hagin repetits
    child1 = parent1[:midpoint] + [gene for gene in parent2 if gene not in parent1[:midpoint]]
    child2 = parent2[:midpoint] + [gene for gene in parent1 if gene not in parent2[:midpoint]]

    return child1, child2

# Funció que reprodueix una població agafan pares de dos en dos i creuant-los. Retorna una nova generació
def crossover_population(population):
    new_gen = []
    for i in range(0, len(population), 2):
        parent1 = population[i]
        parent2 = population[i+1]
        child1, child2 = crossover(parent1, parent2)
        new_gen.append(child1)
        new_gen.append(child2)
    
    return new_gen

crossover_population(_population)


[[8, 2, 4, 7, 5, 6, 1, 3],
 [2, 5, 6, 1, 8, 4, 7, 3],
 [1, 6, 3, 4, 5, 7, 8, 2],
 [5, 7, 1, 4, 6, 3, 8, 2],
 [2, 3, 4, 7, 5, 1, 6, 8],
 [5, 2, 7, 1, 3, 4, 8, 6],
 [4, 6, 5, 8, 3, 7, 1, 2],
 [3, 7, 4, 1, 6, 5, 8, 2],
 [4, 1, 2, 8, 6, 5, 7, 3],
 [8, 2, 4, 6, 1, 5, 3, 7]]

### definir mutació


In [13]:
def mutation(agent):
    # Seleccionar dos posicions aleatòries del gen
    pos1, pos2 = random.sample(range(len(agent)), 2)

    # Intercanviar els elements en les posicions seleccionades
    agent[pos1], agent[pos2] = agent[pos2], agent[pos1]

    return agent


def mutation_with_chance(agent, mutation_chance):

    # Generar un número aleatori per veure si mutem
    if random.random() < mutation_chance:
        return mutation(agent)
    return agent

mutation_with_chance(_agent, MUTATION_CHANCE)

[8, 2, 4, 7, 6, 3, 1, 5]

### definir crossover de tota la població amb mutació

In [14]:
# Funció que reprodueix una població i aplica la mutació als seus fills
def replicate(population, mutation_chance):

    # Nova població
    new_population = crossover_population(population)

    mutated_population = []
    population_len = len(new_population)

    for i in range(population_len):
        # Agafem cada gen de la població i li apliquem la mutació segons el percentatge
        mutated_agent = mutation_with_chance(new_population[i], mutation_chance)
        mutated_population.append(mutated_agent)
    
    return mutated_population

replicate(_population, MUTATION_CHANCE)

[[8, 2, 4, 7, 5, 6, 1, 3],
 [2, 5, 6, 1, 8, 4, 7, 3],
 [1, 6, 3, 4, 5, 7, 8, 2],
 [5, 7, 1, 4, 6, 3, 8, 2],
 [2, 3, 4, 7, 5, 1, 6, 8],
 [5, 2, 7, 1, 3, 4, 8, 6],
 [4, 6, 5, 8, 3, 7, 1, 2],
 [3, 7, 4, 1, 6, 5, 8, 2],
 [4, 1, 2, 8, 6, 5, 7, 3],
 [8, 2, 4, 6, 1, 5, 3, 7]]

### definir fitness function

mirar diagonals i treure repetides

In [15]:
# Funció que comprova si hi ha colisions diagonals entre reines
def fitness_agent(agent):
    colisions = 0

    # Eix Y de la primera reina
    yq1 = 0

    for xq1 in agent:
        yq2 = 0
        for xq2 in agent:
            x = xq2 - xq1
            y = yq2 - yq1

            # Al ser una diagonal, si la X té el mateix valor que la Y, vol dir que colisionen
            if abs(x) == abs(y):
                colisions +=1
            yq2 += 1
        yq1 += 1
    return int((colisions - len(agent))/2)
            

def fitness_population(population):
    fit = []
    for agent in population:
        fit.append(fitness_agent(agent))

    return fit

print("Fitness del gen: ", fitness_agent(_agent))

print("Fitness de la població: ", fitness_population(_population))

Fitness del gen:  4
Fitness de la població:  [4, 2, 10, 3, 5, 3, 11, 1, 4, 3]


### Population Selection (by score)


In [16]:
def sort_population(population, fitness_list):
    llista = list(zip(population, fitness_list))
    llista_sorted = sorted(llista, key=lambda x: x[1], reverse=False)
    return [x[0] for x in llista_sorted]

def select_population(population, fitness_score, population_size):
    sorted_population = sort_population(population, fitness_score)
    return sorted_population[:population_size]


### definir main

(seguir els pasos de classe)

In [17]:
def main(population_size, board_size, mutation_chance):
    population = init_population(population_size, board_size)
    generation = 0

    while True:
        fitness_list = fitness_population(population)
        best_fitness = min(fitness_list)
        best_gen = population[fitness_list.index(best_fitness)]

        if generation % 100 == 0:
            print(f"Generation {generation}: ", best_fitness)

        if best_fitness == 0:
            print(f"Solució trobada a la generació {generation}: ", best_gen)
            break


        # replicate
        new_population = replicate(population, mutation_chance)
        # fitness
        new_fitness_list = fitness_population(new_population)

        combined_population = population + new_population
        combined_fitness_list = fitness_list + new_fitness_list

        # sort + select
        selected_population = select_population(combined_population, combined_fitness_list, population_size)

        # new gen
        population = selected_population
        generation += 1

    return best_gen

### Provar el codi

In [18]:
POPULATION_SIZE = 100
BOARD_SIZE = 30
MUTATION_CHANCE = 0.2

agent = main(POPULATION_SIZE, BOARD_SIZE, MUTATION_CHANCE)
display_agent(agent)

Generation 0:  10
Solució trobada a la generació 71:  [21, 26, 18, 12, 7, 20, 13, 25, 2, 19, 16, 1, 29, 5, 3, 8, 24, 30, 15, 10, 4, 9, 17, 22, 28, 11, 27, 23, 6, 14]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0,